# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending 645-675 per student actually underperformed compared to schools with smaller budgets (585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [13]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [14]:
# Calculate the total number of unique schools
school_count = len(school_data_complete['school_name'].unique())

# Calculate the total number of students
student_count = len(school_data_complete['student_name'])

# Calculate the total budget
total_budget = sum(school_data_complete['budget'].unique())

# Calculate the average (mean) math score
average_math_score = sum(school_data_complete['math_score']) / len(school_data_complete['math_score'])

# Calculate the average (mean) reading score
average_reading_score = sum(school_data_complete['reading_score']) / len(school_data_complete['reading_score'])

# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete['math_score'] >= 70)].count()['student_name']
passing_math_percentage = passing_math_count / float(student_count) * 100

# Calculate the percentage of students who passeed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete['reading_score'] >= 70)].count()['student_name']
passing_reading_percentage = passing_reading_count / float(student_count) * 100

# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]

overall_passing_rate = (passing_math_percentage + passing_reading_percentage)/2

In [15]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({
    "Number of Schools": [school_count],
    "Total Students": [student_count],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math_score],
    "Passing Math Percentage": [passing_math_percentage],
    "Average Reading Score": [average_reading_score],
    "Passing Reading Percentage": [passing_reading_percentage],
    "Overall Passing Rate": [overall_passing_rate]    
    }
)

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary["Passing Math Percentage"] = district_summary["Passing Math Percentage"].round(2)
district_summary["Passing Reading Percentage"] = district_summary["Passing Reading Percentage"].round(2)
district_summary["Overall Passing Rate"] = district_summary["Overall Passing Rate"].round(2)


# Display the DataFrame
district_summary

,Number of Schools,Total Students,Total Budget,Average Math Score,Passing Math Percentage,Average Reading Score,Passing Reading Percentage,Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,74.98,81.87784,85.81,80.39


## School Summary

In [44]:
# School Name
grouped_school_df = school_data_complete.groupby(["school_name"])
# School Type
school_type = grouped_school_df["type"].first()
# Total Students
total_students = grouped_school_df["Student ID"].count()
# Total School Budget
total_school_budget = grouped_school_df["budget"].first()
# Per Student Budget
per_student_budget = total_school_budget / total_students
# Average Math Score
average_math_score = grouped_school_df["math_score"].mean()
# Average Reading Score
average_reading_score = grouped_school_df["reading_score"].mean()
# % Passing Math
passing_math = school_data_complete[school_data_complete["math_score"] >= 70].groupby(["school_name"])["math_score"].count()
percentage_passing_math = passing_math / total_students * 100
# % Passing Reading
passing_read = school_data_complete[school_data_complete["reading_score"] >= 70].groupby(["school_name"])["reading_score"].count()
percentage_passing_reading = passing_read / total_students * 100
# Overall Passing Rate (Average of the above two)
percentage_overall_passing_rate = (percentage_passing_math + percentage_passing_reading) / 2


In [45]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
school_summary_df = pd.DataFrame({"School Type": school_type,
      "Total Students": total_students,
      "Total School Budget": total_school_budget,
      "Per Student Budget": per_student_budget,
      "Average Math Score": average_math_score,
      "Average Reading Score": average_reading_score,
      "% Passing Math": percentage_passing_math,
      "% Passing Reading": percentage_passing_reading,
      "% Overall Passing Rate": percentage_overall_passing_rate})

# Formatting
school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].map("${:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].map("${:,.2f}".format)
# Display the DataFrame
school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992


## Highest-Performing Schools (by % Overall Passing)

In [ ]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.


## Bottom Performing Schools (By % Overall Passing)

In [ ]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.


## Math Scores by Grade

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = 
tenth_graders_scores = 
eleventh_graders_scores = 
twelfth_graders_scores = 

# Use the code to select only the `math_score`.
ninth_grade_math_scores = ninth_graders_scores["math_score"]
tenth_grader_math_scores = tenth_graders_scores["math_score"]
eleventh_grader_math_scores = eleventh_graders_scores.mean()["math_score"]
twelfth_grader_math_scores = twelfth_graders_scores["math_score"]

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`


# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

## Reading Score by Grade 

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = 
tenth_graders_scores = 
eleventh_graders_scores = 
twelfth_graders_scores = 

# Use the code to select only the `reading_score`.
ninth_grade_reading_scores = ninth_graders_scores["reading_score"]
tenth_grader_reading_scores = tenth_graders_scores["reading_score"]
eleventh_grader_reading_scores = eleventh_graders_scores.mean()["reading_score"]
twelfth_grader_reading_scores = twelfth_graders_scores["reading_score"]

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`


# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

## Scores by School Spending

In [ ]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [ ]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [ ]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = 

In [ ]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

In [ ]:
# Assemble into DataFrame
spending_summary = 

# Display results
spending_summary

## Scores by School Size

In [ ]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = 

In [ ]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary.groupby(["School Size"]).mean()["% Overall Passing"]

In [ ]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`


# Display results
size_summary

## Scores by School Type

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = 
type_reading_scores = 
type_passing_math = 
type_passing_reading = 
type_overall_passing = 

# Use the code provided to select new column data
average_math_score_by_type = type_math_scores["Average Math Score"]
average_reading_score_by_type = type_reading_scores["Average Reading Score"]
average_percent_passing_math_by_type = type_passing_math["% Passing Math"]
average_percent_passing_reading_by_type = type_passing_reading["% Passing Reading"]
average_percent_overall_passing_by_type = type_overall_passing["% Overall Passing"]

In [ ]:
# Assemble the new data by type into a DataFrame called `type_summary`


# Display results
type_summary